# imports

In [2]:
import pickle
from srgnn_pl import SRGNN_model, SRGNN_Map_Dataset, SRGNN_sampler
from utils import fake_parser
import torch
import os

from torch.utils.data import DataLoader
import pytorch_lightning as pl

import numpy as np

In [3]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import yaml
from math import ceil
from sklearn.mixture import GaussianMixture
from IPython.display import clear_output

In [4]:
torch.set_float32_matmul_precision('medium')

# data loading

In [37]:
#global_run_id='run-20240209_162656-h23ej73g'

finetuned_run_id='run-20240213_043223-0zuvfc9x'
global_run_id=finetuned_run_id

In [38]:
with open(f"./wandb/{global_run_id}/files/config.yaml", "r") as stream:
        config=yaml.safe_load(stream)

keys=list(config.keys())
for k in keys:
    if k not in fake_parser().__dict__.keys():
        del config[k]
    else:
        config[k]=config[k]['value']

opt=fake_parser(**config)
print(opt.__dict__)

model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{global_run_id.split('-')[-1]}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{global_run_id.split('-')[-1]}/checkpoints/")[0], opt=opt)

{'dataset': 'yoochoose_custom', 'batchSize': 128, 'hiddenSize': 64, 'epoch': 60, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 3, 'patience': 6, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.2, 'pretrained_embedings': True, 'unfreeze_epoch': 2}


In [39]:
#with open(f'./gmm_all_32_k-means++_{opt.hiddenSize}.gmm', 'rb') as gmm_file:
with open(f'./gmm_better_32_k-means++_{opt.hiddenSize}.gmm', 'rb') as gmm_file:
    gm=pickle.load(gmm_file)
#    session_labels=gm.predict(session_emb)

In [40]:
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)
test_dataloader=DataLoader(test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(test_dataset, opt.batchSize, shuffle=False, drop_last=False),
                             drop_last=False
                            )

data masking start
data masking 1
data masking 2
data masking 3
done masking


# evaluate each cluster model

## all sessions

In [41]:
session_emb=[]

model.to('cuda')
for batch in tqdm(test_dataloader, total=test_dataset.length//opt.batchSize):
    batch=[b.to('cuda') for b in batch]
    session_emb.append(model.get_session_embeddings(batch).cpu().detach().numpy())
model.to('cpu')
session_emb=np.concatenate(session_emb)

1214it [00:08, 137.23it/s]                          


In [42]:
session_labels=[]
for i in tqdm(range(ceil(session_emb.shape[0]/opt.batchSize))):
    session_labels.append(gm.predict(session_emb[i*opt.batchSize: (i+1)*opt.batchSize]))
session_labels=np.concatenate(session_labels)

100%|██████████| 1214/1214 [00:01<00:00, 826.39it/s]


In [43]:
hit,mrr=[],[]
model.to('cuda')
for batch in tqdm(test_dataloader, total=test_dataset.length//opt.batchSize):
    batch=[b.to('cuda') for b in batch]
    sub_scores, targets=model.predict_step(batch)
    targets=targets.flatten()
    for score, target in zip(sub_scores, targets):
        correct_pred=torch.isin(target - 1, score).cpu()
        hit.append(correct_pred)
        if not correct_pred:
            mrr.append(0)
        else:
            mrr.append(1 / (torch.where(score == target - 1)[0][0] + 1).cpu())
model.to('cpu')
hit=np.array(hit)
mrr=np.array(mrr)

1214it [00:54, 22.35it/s]                          


### trained from scratch

In [28]:
runs_df=pd.read_csv('./wandb_export_raw_all.csv')
runs_df['cluster']=runs_df.Name.map(lambda x: int(x.split('_')[-1]))

In [29]:
cluster_results=[]
cluster_results.append(
{'cluster':-1, 
 'test_loss': np.nan,
 'test_hit': np.average(hit),
 'test_mrr': np.average(mrr)})

for c in tqdm(range(gm.n_components)):
    clear_output(wait=True)
    idxs=np.arange(len(test_data[0]))[session_labels==c]

    cluster_sess=[test_data[0][i] for i in idxs]
    cluster_targets=[test_data[1][i] for i in idxs]
    cluster_data=(cluster_sess,cluster_targets)
    cluster_dataset=SRGNN_Map_Dataset(cluster_data, shuffle=False)
    cluster_dataloader=DataLoader(cluster_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(cluster_dataset, opt.batchSize, shuffle=False, drop_last=False),
                             drop_last=False
                            )
    run_id=runs_df.loc[runs_df.cluster==c].ID.item()
    cluster_model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{run_id}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{run_id}/checkpoints/")[0], opt=opt)
    
    trainer=pl.Trainer(limit_test_batches=ceil(cluster_dataset.length/opt.batchSize),limit_predict_batches=ceil(cluster_dataset.length/opt.batchSize))
    metrics=trainer.test(cluster_model, cluster_dataloader)[0]#only one dataloader
    metrics['cluster']=c
    metrics['global_model_hit']=100*np.average(hit[idxs])
    metrics['global_model_mrr']=100*np.average(mrr[idxs])
    cluster_results.append(metrics)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data masking start
data masking 1
data masking 2
data masking 3
done masking


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_hit          │     63.52531051635742     │
│         test_loss         │     5.713991641998291     │
│         test_mrr          │    26.996490478515625     │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 32/32 [01:20<00:00,  2.50s/it]


In [30]:
cluster_results=pd.DataFrame(cluster_results)
cluster_results

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr
0,-1,NaN,0.553531,0.248305,NaN,NaN
1,0,6.621505,51.548405,23.252512,53.329782,24.557642
2,1,6.454111,45.515423,18.264372,47.145589,19.081339
3,2,6.628732,40.922070,14.518913,44.805195,16.413542
4,3,6.079962,63.917828,30.493874,64.709763,30.852930
5,4,8.367682,45.151176,20.737585,51.160433,22.815957
6,5,5.501886,68.205498,35.135696,70.256024,34.819782
7,6,6.545033,54.092262,23.490585,57.482599,25.490220
8,7,5.627601,64.603004,26.756392,66.477987,28.742248
9,8,5.816460,60.559658,25.042292,63.295019,26.194494


In [32]:
(cluster_results.test_hit>cluster_results.global_model_hit).any(), (cluster_results.test_mrr>cluster_results.global_model_mrr).any()

(False, True)

In [34]:
cluster_results.iloc[np.arange(cluster_results.shape[0])[cluster_results.test_mrr>cluster_results.global_model_mrr]]

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr
6,5,5.501886,68.205498,35.135696,70.256024,34.819782


In [36]:
cluster_results.loc[cluster_results.global_model_hit<100*np.average(hit)]

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr
1,0,6.621505,51.548405,23.252512,53.329782,24.557642
2,1,6.454111,45.515423,18.264372,47.145589,19.081339
3,2,6.628732,40.922070,14.518913,44.805195,16.413542
5,4,8.367682,45.151176,20.737585,51.160433,22.815957
13,12,6.043377,54.361897,22.599600,55.291622,23.257653
16,15,7.424316,36.012962,16.525658,42.018538,19.445396
17,16,9.478106,28.735292,10.815783,35.063694,14.230315
18,17,6.353772,48.304508,22.010084,50.141163,22.443078
21,20,8.149501,38.733315,16.378510,51.173896,22.492508
24,23,12.635973,14.129725,4.684999,21.082391,7.414331


## full sessions

In [124]:
full_session_emb=[]
full_sessions=[]
for batch in tqdm(test_dataloader, total=test_dataset.length//opt.batchSize):
    # use only original sessions
    a=batch[3][0]
    a=torch.vstack([torch.zeros(a.shape[1]), a])
    idxs=torch.diff(a, dim=0).sum(axis=1)
    for i in range(len(batch)):
        batch[i]=batch[i][:,idxs>0]
    #print(batch[0].shape)
    #break
    full_sessions.append(batch)
    full_session_emb.append(model.get_session_embeddings(batch).detach().numpy())
full_session_emb=np.concatenate(full_session_emb)

152it [00:13, 11.67it/s]                         


In [125]:
full_session_labels=gm.predict(full_session_emb)

In [126]:
len(full_session_labels)

34034

In [127]:
hit,mrr=[],[]
for batch in tqdm(full_sessions):
    sub_scores, targets=model.predict_step(batch)
    targets=targets.flatten()
    for score, target in zip(sub_scores, targets):
        correct_pred=torch.isin(target - 1, score)
        hit.append(correct_pred)
        if not correct_pred:
            mrr.append(0)
        else:
            mrr.append(1 / (torch.where(score == target - 1)[0][0] + 1))
hit=np.array(hit)
mrr=np.array(mrr)

100%|██████████| 152/152 [00:12<00:00, 12.41it/s]


In [129]:
full_sess_results=[]
full_sess_results.append(
{'cluster':-1, 
 'global_model_hit': np.average(hit),
 'global_model_mrr': np.average(mrr)})

for c in tqdm(range(gm.n_components)):
    clear_output(wait=True)
    idxs=full_session_labels==c

    run_id=runs_df.loc[runs_df.cluster==c].ID.item()
    cluster_model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{run_id}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{run_id}/checkpoints/")[0], opt=opt)
    cluster_model.to('cpu')
    
    chit,cmrr=[],[]
    counter=0
    prev_batch=None
    for batch in (full_sessions):
        batch_idxs=idxs[counter:counter+batch[4].shape[-1]]
        counter+=batch[4].shape[-1]

        batch=[b[:, batch_idxs] for b in batch]
        

        sub_scores, targets=cluster_model.predict_step(batch)
        targets=targets.flatten()
        for score, target in zip(sub_scores, targets):
            correct_pred=torch.isin(target - 1, score)
            chit.append(correct_pred)
            if not correct_pred:
                cmrr.append(0)
            else:
                cmrr.append(1 / (torch.where(score == target - 1)[0][0] + 1))

    metrics={}
    metrics['cluster']=c
    metrics['model_hit']=100*np.average(chit)
    metrics['model_mrr']=100*np.average(cmrr)
    metrics['global_model_hit']=100*np.average(hit[idxs])
    metrics['global_model_mrr']=100*np.average(mrr[idxs])
    full_sess_results.append(metrics)

100%|██████████| 32/32 [00:16<00:00,  1.90it/s]


In [131]:
full_sess_results=pd.DataFrame(full_sess_results)

In [136]:
full_sess_results.iloc[(full_sess_results.model_mrr>full_sess_results.global_model_mrr).values]

,cluster,test_loss,test_hit,test_mrr,model_hit,model_mrr,global_model_hit,global_model_mrr
10,9,NaN,NaN,NaN,61.501597,28.099945,62.616822,27.867965
15,14,NaN,NaN,NaN,68.902439,35.281173,68.471954,34.602202


In [135]:
full_sess_results.iloc[(full_sess_results.model_hit>full_sess_results.global_model_hit).values]

,cluster,test_loss,test_hit,test_mrr,model_hit,model_mrr,global_model_hit,global_model_mrr
15,14,NaN,NaN,NaN,68.902439,35.281173,68.471954,34.602202
26,25,NaN,NaN,NaN,63.004172,24.414224,62.054795,24.630326


# finetuned

## all sessions

In [79]:
runs_df=pd.read_csv('./wandb_export_gm_finetune_runs.csv')
runs_df['cluster']=runs_df.Name.map(lambda x: int(x.split('_')[-1]))

In [80]:
cluster_results=[]
cluster_results.append(
{'cluster':-1, 
 'test_loss': np.nan,
 'test_hit': np.average(hit),
 'test_mrr': np.average(mrr)})

for c in tqdm(range(gm.n_components)):
    clear_output(wait=True)
    if runs_df.loc[runs_df.cluster==c].empty:
        continue
    idxs=np.arange(len(test_data[0]))[session_labels==c]

    cluster_sess=[test_data[0][i] for i in idxs]
    cluster_targets=[test_data[1][i] for i in idxs]
    cluster_data=(cluster_sess,cluster_targets)
    cluster_dataset=SRGNN_Map_Dataset(cluster_data, shuffle=False)
    cluster_dataloader=DataLoader(cluster_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(cluster_dataset, opt.batchSize, shuffle=False, drop_last=False),
                             drop_last=False
                            )
    run_id=runs_df.loc[runs_df.cluster==c].ID.item()
    cluster_model=SRGNN_model.load_from_checkpoint(f"./GNN_master/{run_id}/checkpoints/"+
                                       os.listdir(f"./GNN_master/{run_id}/checkpoints/")[0], opt=opt)
    
    trainer=pl.Trainer(limit_test_batches=ceil(cluster_dataset.length/opt.batchSize),limit_predict_batches=ceil(cluster_dataset.length/opt.batchSize))
    metrics=trainer.test(cluster_model, cluster_dataloader)[0]#only one dataloader
    metrics['cluster']=c
    metrics['global_model_hit']=100*np.average(hit[idxs])
    metrics['global_model_mrr']=100*np.average(mrr[idxs])
    cluster_results.append(metrics)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data masking start
data masking 1
data masking 2
data masking 3
done masking


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_hit          │     35.08748245239258     │
│         test_loss         │     8.98015308380127      │
│         test_mrr          │    15.542804718017578     │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 32/32 [01:30<00:00,  2.83s/it]


In [81]:
cluster_results=pd.DataFrame(cluster_results)
cluster_results=cluster_results.merge(pd.DataFrame(np.unique(session_labels, return_counts=True)).T.rename(columns={0:'cluster',1:'size'}),
                      on='cluster')

In [82]:
(cluster_results.test_hit>cluster_results.global_model_hit).any(), (cluster_results.test_mrr>cluster_results.global_model_mrr).any()

(True, True)

In [83]:
cluster_results.iloc[np.arange(cluster_results.shape[0])[cluster_results.test_mrr>cluster_results.global_model_mrr]]

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr,size
0,0,6.345438,46.706402,17.293476,47.368421,17.076397,5909
3,3,9.786974,32.537613,13.866294,30.728004,13.043829,2239
5,5,5.847559,61.718750,33.115646,62.188057,32.829809,4488
19,20,1.729631,92.307693,92.307693,92.307692,92.307692,52
24,25,5.962016,54.421932,26.047878,54.209524,25.839076,2625
28,29,6.521373,44.465172,16.526772,43.761028,16.370745,3967


In [84]:
cluster_results.iloc[np.arange(cluster_results.shape[0])[cluster_results.test_hit>cluster_results.global_model_hit]]

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr,size
3,3,9.786974,32.537613,13.866294,30.728004,13.043829,2239
19,20,1.729631,92.307693,92.307693,92.307692,92.307692,52
23,24,13.693072,18.772322,7.410641,17.639594,8.191401,788
24,25,5.962016,54.421932,26.047878,54.209524,25.839076,2625
27,28,6.638066,46.627441,21.624046,46.352941,21.992768,3400
28,29,6.521373,44.465172,16.526772,43.761028,16.370745,3967


In [85]:
session_labels.shape

(155292,)

In [86]:
cluster_results.loc[cluster_results.global_model_hit<np.average(hit)*100/1.25]

,cluster,test_loss,test_hit,test_mrr,global_model_hit,global_model_mrr,size
3,3,9.786974,32.537613,13.866294,30.728004,13.043829,2239
23,24,13.693072,18.772322,7.410641,17.639594,8.191401,788
28,29,6.521373,44.465172,16.526772,43.761028,16.370745,3967
30,31,8.980153,35.087482,15.542805,36.197637,15.979771,9310
